# GLACIS Integration Demo: Attestation for AI

**What is GLACIS?** A library that creates tamper-proof receipts for AI operations.
Prove what your AI did, what data it saw - without sensitive data leaving your environment.

## What You'll Learn
1. **Offline Mode** - Self-signed attestations, no API key needed
2. **OpenAI Integration** - Automatic attestation for OpenAI API calls
3. **Azure AI Integration** - Automatic attestation for Azure AI Inference calls

## Core Concept: "Hash Locally, Prove Globally"
- Input/output data is hashed locally (SHA-256)
- Only hashes are transmitted, never actual payloads
- Receipts are cryptographically signed (Ed25519)

## Setup

Install glacis with the integrations you need:

```bash
pip install glacis              # Core library
pip install glacis[openai]      # OpenAI integration
pip install glacis[azure]       # Azure AI Inference integration
pip install glacis[openai,azure] # Both integrations
```

In [ ]:
import os
from pathlib import Path

# Load environment variables from .env file if present
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

# Generate a 32-byte signing seed for Ed25519
# In production, persist this securely to maintain the same identity
SIGNING_SEED = os.urandom(32)
print(f"Generated signing seed: {SIGNING_SEED.hex()[:16]}...")

---
## 1. Offline Mode (No API Key Required)

Perfect for development and testing. Receipts are self-signed and stored locally.
Witness status will be `UNVERIFIED` until submitted to the transparency log.

In [ ]:
from glacis import Glacis

# Create an offline Glacis client
glacis_offline = Glacis(mode="offline", signing_seed=SIGNING_SEED)
print(f"Mode: {glacis_offline.mode}")

In [ ]:
# Simulate an AI interaction
prompt = "What is the capital of France?"
response = "The capital of France is Paris."

# Create an attestation
# Input and output are hashed locally, never sent anywhere
receipt = glacis_offline.attest(
    service_id="demo-app",
    operation_type="inference",
    input={"prompt": prompt},
    output={"response": response},
    metadata={"model": "gpt-4", "temperature": 0.7},
)

print("Attestation created!")
print(f"  Receipt ID: {receipt.attestation_id}")
print(f"  Payload Hash: {receipt.payload_hash}")
print(f"  Signature: {receipt.signature[:40]}...")
print(f"  Witness Status: {receipt.witness_status}")

In [ ]:
# Verify the receipt
result = glacis_offline.verify(receipt)

print("Verification result:")
print(f"  Signature Valid: {result.signature_valid}")
print(f"  Overall Valid: {result.valid}")
print(f"  Witness Status: {result.witness_status}")

# Receipts are stored locally at ~/.glacis/receipts.db
print(f"\nReceipts stored in: ~/.glacis/receipts.db")

---
## 2. Online Mode (API Key Required)

For production use. Attestations are witnessed by the Glacis server and included in a Merkle tree.
This provides third-party verifiability and tamper-evident logs.

**Get your API key at https://glacis.io**

In [ ]:
# Set your API key (or set GLACIS_API_KEY environment variable)
GLACIS_API_KEY = os.environ.get("GLACIS_API_KEY", "")

if not GLACIS_API_KEY:
    print("No GLACIS_API_KEY found. Skipping online mode demo.")
    print("Set GLACIS_API_KEY environment variable or get a key at https://glacis.io")
    glacis_online = None
else:
    glacis_online = Glacis(api_key=GLACIS_API_KEY)
    print(f"Mode: {glacis_online.mode}")
    print("Online client ready!")

In [ ]:
if glacis_online:
    # Create a witnessed attestation
    online_receipt = glacis_online.attest(
        service_id="demo-app",
        operation_type="inference",
        input={"prompt": "What is 2 + 2?"},
        output={"response": "4"},
    )
    
    print("Online attestation created!")
    print(f"  Receipt ID: {online_receipt.attestation_id}")
    print(f"  Leaf Index: {online_receipt.leaf_index}")  # Position in Merkle tree
    print(f"  Badge URL: {online_receipt.badge_url}")    # Shareable verification link
    
    # Access Merkle proof details if available
    if online_receipt.receipt and online_receipt.receipt.transparency_proofs:
        root_hash = online_receipt.receipt.transparency_proofs.sth_curr.root_hash
        print(f"  Merkle Root: {root_hash[:32]}...")
    
    print(f"  Witness Status: WITNESSED")
else:
    print("Skipped - no API key")

---
## 3. OpenAI Integration

Automatically attest every OpenAI API call with a drop-in wrapper.

**Requires:** `pip install glacis[openai]` and `OPENAI_API_KEY`

In [ ]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")

try:
    from glacis.integrations.openai import attested_openai, get_last_receipt as get_openai_receipt
    openai_available = True
except ImportError:
    openai_available = False
    print("OpenAI integration not installed. Run: pip install glacis[openai]")

if openai_available and not OPENAI_API_KEY:
    print("No OPENAI_API_KEY found. Skipping OpenAI demo.")
    print("Set OPENAI_API_KEY environment variable to enable this section.")

### 3a. OpenAI + Offline Attestation

No GLACIS API key needed - receipts are signed locally.

In [ ]:
if openai_available and OPENAI_API_KEY:
    # Create an attested OpenAI client (offline mode)
    openai_client_offline = attested_openai(
        openai_api_key=OPENAI_API_KEY,
        offline=True,
        signing_seed=SIGNING_SEED,
        service_id="openai-demo-offline",
    )
    print("Attested OpenAI client ready (offline mode)!")
    
    # Make an API call - attestation happens automatically
    response = openai_client_offline.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "What is 2 + 2? Answer in one word."}],
        max_tokens=10,
    )
    
    print(f"\nResponse: {response.choices[0].message.content}")
    
    # Get the attestation receipt
    openai_receipt_offline = get_openai_receipt()
    print(f"\nAttestation created automatically!")
    print(f"  Receipt ID: {openai_receipt_offline.attestation_id}")
    print(f"  Payload Hash: {openai_receipt_offline.payload_hash}")
    print(f"  Witness Status: {openai_receipt_offline.witness_status}")
else:
    print("Skipped - OpenAI not available or no API key")

### 3b. OpenAI + Online Attestation

Requires both GLACIS and OpenAI API keys. Receipts are witnessed by the GLACIS server.

In [ ]:
if openai_available and OPENAI_API_KEY and GLACIS_API_KEY:
    # Create an attested OpenAI client (online mode)
    openai_client_online = attested_openai(
        glacis_api_key=GLACIS_API_KEY,
        openai_api_key=OPENAI_API_KEY,
        service_id="openai-demo-online",
    )
    print("Attested OpenAI client ready (online mode)!")
    
    # Make an API call - attestation happens automatically
    response = openai_client_online.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "What is the square root of 16?"}],
        max_tokens=10,
    )
    
    print(f"\nResponse: {response.choices[0].message.content}")
    
    # Get the attestation receipt
    openai_receipt_online = get_openai_receipt()
    print(f"\nAttestation created automatically!")
    print(f"  Receipt ID: {openai_receipt_online.attestation_id}")
    print(f"  Leaf Index: {openai_receipt_online.leaf_index}")
    print(f"  Badge URL: {openai_receipt_online.badge_url}")
    
    # Access Merkle proof
    if openai_receipt_online.receipt and openai_receipt_online.receipt.transparency_proofs:
        root_hash = openai_receipt_online.receipt.transparency_proofs.sth_curr.root_hash
        print(f"  Merkle Root: {root_hash[:32]}...")
elif openai_available and OPENAI_API_KEY:
    print("Skipped - no GLACIS_API_KEY. Set it for online attestation.")
else:
    print("Skipped - OpenAI not available or no API keys")

---
## 4. Azure AI Integration

Automatically attest every Azure AI Inference call with a drop-in wrapper.

**Requires:** `pip install glacis[azure]` and Azure credentials

In [ ]:
AZURE_ENDPOINT = os.environ.get("AZURE_AI_ENDPOINT", "")  # e.g., https://my-endpoint.inference.ai.azure.com
AZURE_API_KEY = os.environ.get("AZURE_AI_API_KEY", "")

try:
    from azure.ai.inference.models import UserMessage
    from azure.core.credentials import AzureKeyCredential
    from glacis.integrations.azure import attested_azure_inference, get_last_receipt as get_azure_receipt
    azure_available = True
except ImportError:
    azure_available = False
    print("Azure AI integration not installed. Run: pip install glacis[azure]")

if azure_available and (not AZURE_ENDPOINT or not AZURE_API_KEY):
    print("No Azure credentials found. Skipping Azure AI demo.")
    print("Set AZURE_AI_ENDPOINT and AZURE_AI_API_KEY environment variables to enable this section.")
    print("Example: AZURE_AI_ENDPOINT=https://my-endpoint.inference.ai.azure.com")

### 4a. Azure AI + Offline Attestation

No GLACIS API key needed - receipts are signed locally.

In [ ]:
if azure_available and AZURE_ENDPOINT and AZURE_API_KEY:
    # Create an attested Azure AI client (offline mode)
    azure_client_offline = attested_azure_inference(
        endpoint=AZURE_ENDPOINT,
        credential=AzureKeyCredential(AZURE_API_KEY),
        offline=True,
        signing_seed=SIGNING_SEED,
        service_id="azure-demo-offline",
    )
    print("Attested Azure AI client ready (offline mode)!")
    
    # Make an API call - attestation happens automatically
    response = azure_client_offline.complete(
        messages=[UserMessage(content="What is 3 + 3? Answer in one word.")],
        model="gpt-4o",  # Adjust based on your deployment
        max_tokens=10,
    )
    
    print(f"\nResponse: {response.choices[0].message.content}")
    
    # Get the attestation receipt
    azure_receipt_offline = get_azure_receipt()
    print(f"\nAttestation created automatically!")
    print(f"  Receipt ID: {azure_receipt_offline.attestation_id}")
    print(f"  Payload Hash: {azure_receipt_offline.payload_hash}")
    print(f"  Witness Status: {azure_receipt_offline.witness_status}")
else:
    print("Skipped - Azure AI not available or no credentials")

### 4b. Azure AI + Online Attestation

Requires both GLACIS API key and Azure credentials. Receipts are witnessed by the GLACIS server.

In [ ]:
if azure_available and AZURE_ENDPOINT and AZURE_API_KEY and GLACIS_API_KEY:
    # Create an attested Azure AI client (online mode)
    azure_client_online = attested_azure_inference(
        glacis_api_key=GLACIS_API_KEY,
        endpoint=AZURE_ENDPOINT,
        credential=AzureKeyCredential(AZURE_API_KEY),
        service_id="azure-demo-online",
    )
    print("Attested Azure AI client ready (online mode)!")
    
    # Make an API call - attestation happens automatically
    response = azure_client_online.complete(
        messages=[UserMessage(content="What is the capital of Japan?")],
        model="gpt-4o",  # Adjust based on your deployment
        max_tokens=20,
    )
    
    print(f"\nResponse: {response.choices[0].message.content}")
    
    # Get the attestation receipt
    azure_receipt_online = get_azure_receipt()
    print(f"\nAttestation created automatically!")
    print(f"  Receipt ID: {azure_receipt_online.attestation_id}")
    print(f"  Leaf Index: {azure_receipt_online.leaf_index}")
    print(f"  Badge URL: {azure_receipt_online.badge_url}")
    
    # Access Merkle proof
    if azure_receipt_online.receipt and azure_receipt_online.receipt.transparency_proofs:
        root_hash = azure_receipt_online.receipt.transparency_proofs.sth_curr.root_hash
        print(f"  Merkle Root: {root_hash[:32]}...")
elif azure_available and AZURE_ENDPOINT and AZURE_API_KEY:
    print("Skipped - no GLACIS_API_KEY. Set it for online attestation.")
else:
    print("Skipped - Azure AI not available or no credentials")

---
## Summary

| Integration | Mode | API Keys Required | Witness Status |
|------------|------|-------------------|----------------|
| Core | Offline | None | `UNVERIFIED` |
| Core | Online | GLACIS | `WITNESSED` |
| OpenAI | Offline | OpenAI | `UNVERIFIED` |
| OpenAI | Online | GLACIS + OpenAI | `WITNESSED` |
| Azure AI | Offline | Azure | `UNVERIFIED` |
| Azure AI | Online | GLACIS + Azure | `WITNESSED` |

### What's in a Receipt?

**Offline Receipt (`OfflineAttestReceipt`):**
- `attestation_id`: Unique identifier (`oatt_xxx`)
- `payload_hash`: SHA-256 of canonical JSON (input + output)
- `signature`: Ed25519 signature over the attestation
- `timestamp`: When the attestation was created
- `witness_status`: Always `UNVERIFIED`

**Online Receipt (`AttestReceipt`):**
- `attestation_id`: Unique identifier (`att_xxx`)
- `attestation_hash`: Hash of the attestation content
- `leaf_index`: Position in Merkle tree
- `tree_size`: Tree size when receipt was issued
- `badge_url`: Shareable verification link
- `receipt.transparency_proofs`: Full Merkle inclusion proof

### Environment Variables Reference

Create a `.env` file with these values:

```bash
# GLACIS (required for online attestation)
GLACIS_API_KEY=glsk_live_xxx

# OpenAI (required for OpenAI integration)
OPENAI_API_KEY=sk-xxx

# Azure AI (required for Azure integration)
AZURE_AI_ENDPOINT=https://my-endpoint.inference.ai.azure.com
AZURE_AI_API_KEY=xxx
```

### Next Steps

1. **Get an API key** at https://glacis.io for production use
2. **Explore the Anthropic integration**: `from glacis.integrations.anthropic import attested_anthropic`
3. **Query the transparency log**: `glacis.query_log(org_id="...", service_id="...")`
4. **Check stored receipts**: `~/.glacis/receipts.db` (SQLite)

In [ ]:
# Cleanup
glacis_offline.close()
if glacis_online:
    glacis_online.close()
print("Done!")